# Assignment 1: Part Of Speech tagging

## Imports

In [ ]:
# Dependency management
!pip install tensorflow==2.7.0 numpy==1.21.4 pandas==1.3.4 requests==2.26.0 gensim==4.1.2 wandb==0.12.7 -qqq

In [ ]:
# Disable tensorflow warnings
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

tf.autograph.set_verbosity(0)

import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [ ]:
# Text pre-processing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Model definition
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, TimeDistributed, Dense

# Data packages
import numpy as np
import pandas as pd

# System packages
import glob

# Cloning
from copy import deepcopy

# File management
import requests
import zipfile
import io
import pathlib

# Notebook visualization
from IPython.core.display import display, HTML

# Typing
from typing import Set

# For GloVe wrapper
from gensim import downloader as gensloader
from gensim.models.keyedvectors import KeyedVectors

# Plotting
import plotly.express as px
import wandb
from wandb.keras import WandbCallback

# Metrics and utility
from sklearn.metrics import confusion_matrix

## 1 - Data Pipeline

### 1.1 - Data loading
First, we load the dataset and store it into a dataframe.

In [ ]:
DATASET_PATH = './dependency_treebank'  # Change if dataset already present locally
DATASET_URL = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip'


def load_dataset(ds_path: str, ds_url: str) -> pd.DataFrame:
    # Check if dataset is already present, otherwise download it
    if not pathlib.Path(ds_path).exists():
        request_zip = requests.get(ds_url, stream=True)
        zip = zipfile.ZipFile(io.BytesIO(request_zip.content))
        zip.extractall()

    # Load each file into a list
    documents = []
    for file_name in sorted(glob.glob(f"{ds_path}/*.dp")):
        with open(file_name) as f:
            documents.append(f.read())

    # Convert each row of the documents into a list
    raw_df = []
    sentence_idx = 0
    for doc_idx, doc in enumerate(documents):
        rows = doc.split('\n')
        for row in rows:
            cols = row.split('\t')[:2]  # Ignore the last column
            if cols == ['']:
                sentence_idx += 1
            else:
                raw_df.append([doc_idx, sentence_idx, *cols])

    # Finally, convert the nested list into a pandas dataframe
    df = pd.DataFrame(raw_df, columns=['document', 'sentence', 'token', 'tag'])
    return df


dataset = load_dataset(DATASET_PATH, DATASET_URL)
dataset.head()

### 1.2 - GloVe loading
Then, we load the GloVe embeddings (GloVe-50, to be precise).

In [ ]:
EMB_DIM = 50
GLOVE_TYPE = f'glove-wiki-gigaword-{EMB_DIM}'
GLOVE_FILE = f'./glove/glove-wiki-gigaword-{EMB_DIM}.kv'


def load_glove(gl_file: str, gl_type: str) -> KeyedVectors:
    # Load local version
    path = pathlib.Path(gl_file)
    if path.exists():
        return KeyedVectors.load(gl_file)

    # Otherwise download and store glove
    path.parent.mkdir(parents=True, exist_ok=True)
    glove = gensloader.load(gl_type)
    glove.save(gl_file)
    return glove


glove = load_glove(GLOVE_FILE, GLOVE_TYPE)

In [ ]:
# Test GloVe loading
print(f'cat = {glove["cat"]}')

### 1.3 - Data visualization
One of the most important ML tasks is getting familiar with the data in order to gain a deeper insight on their structure and nature.

To do so, we define a function that displays tokens with their POS tags in a human-friendlier way.

In [ ]:
# NOTE: this could be put in its own file to keep things clean,
#       but to upload just one notebook we instead code-golfed a bit :)

# Define a mapping between POS tags, their meaning and some colors
from collections import defaultdict

tag_map = {
    'CC': ('Coordin. Conjunction', '#c18401'),
    'TO': ('“to”', '#c18401'),
    'DT': ('Determiner', '#c18401'),
    'UH': ('Interjection', '#c18401'),
    'EX': ('Existential ‘there', '#c18401'),
    'MD': ('Modal can', '#c18401'),
    'LS': ('List item marker', '#c18401'),
    'IN': ('Preposition/sub-conj', '#c18401'),
    'CD': ('Cardinal number', '#282828'),
    'FW': ('Foreign word', '#282828'),
    'NN': ('Noun, singular/mass', '#282828'),
    'NNS': ('Noun, plural', '#282828'),
    'NNP': ('Proper noun, singul.', '#282828'),
    'NNPS': ('Proper noun, plural', '#282828'),
    'JJ': ('Adjective', '#50a14f'),
    'JJR': ('Adj. comparative ', '#50a14f'),
    'JJS': ('Adj. superlative ', '#50a14f'),
    'VB': ('Verb, base form', '#e45649'),
    'VBD': ('Verb, past tense ', '#e45649'),
    'VBG': ('Verb, gerund ', '#e45649'),
    'VBN': ('Verb, past particip. ', '#e45649'),
    'VBP': ('Verb, non-3sg pres', '#e45649'),
    'VBZ': ('Verb, 3sg pres ', '#e45649'),
    'WDT': ('Wh-determiner', '#4078f2'),
    'WP': ('Wh-pronoun', '#4078f2'),
    'WP$': (' Possessive wh-', '#4078f2'),
    'WRB': ('Wh-adverb how', '#4078f2'),
    'PDT': ('Predeterminer ', '#4078f2'),
    'POS': ('Possessive ending', '#4078f2'),
    'PP': ('Personal pronoun', '#4078f2'),
    'PP$': (' Possessive pronoun ', '#4078f2'),
    'RB': ('Adverb', '#a626a4'),
    'RBR': ('Adverb, comparative', '#a626a4'),
    'RBS': ('Adverb, superlative', '#a626a4'),
    'RP': ('Particle', '#a626a4'),
}
tag_map = defaultdict(lambda: ('', '#282828'), tag_map)


def display_pos_tagging(tokens: pd.Series,
                        predicted_tags: pd.Series,
                        correct_tags: pd.Series = None,
                        limit=1000):
    # If no correct tags are passed, we ignore the "error highlighting"
    if correct_tags is None:
        correct_tags = predicted_tags

    # Limit the inputs
    tokens = tokens[:limit]
    predicted_tags = predicted_tags[:limit]
    correct_tags = correct_tags[:limit]

    # Iterate through tokens and tags, generating styled HTML based on the tags
    html_sequence = []
    for token, tag, correct in zip(tokens, predicted_tags, correct_tags):
        tag_meaning = tag_map[tag][0]
        err = 'pos-error' if tag != correct else ''
        h = f'<div class="token {tag} {err}">{token} <span class="tag">[{tag}] {tag_meaning}</span></div>'
        if tag == '.':
            h += '<div class="separator"/>'
        html_sequence.append(h)
    html_body = '<div class="pos-visualizer">'
    html_body += ' '.join(html_sequence) + '</div>'

    # Generate the style (WARNING: CSS lies ahead)
    html_style = """
	<style>
	.pos-visualizer { padding: 32px; background-color: #FEFEFE; border-left:solid 1px grey;}
	.token { position:relative; display:inline-block; font-size:16px;}
	.token .tag { 
		visibility:hidden; width: 120px; text-align:center; position:absolute;
		width: 160px; background-color: #282828; color: #fff; border-radius: 6px;
		z-index: 1; bottom: 100%; left: 50%; margin-left:-80px; font-size:12px;
	}
	.pos-error { text-decoration: underline solid #F94144;}
	.separator { margin-top:12px }
	.token:hover .tag { visibility:visible }
	"""
    html_style += '\n'.join((f'.{tag} {{color:{tag_map[tag][1]};}}'
                             for tag in predicted_tags.unique()))
    html_style += '</style>'

    # Display the HTML in the cell's output
    display(HTML(html_style + html_body))

In [ ]:
# Display some sample POS tagging (hover on text for tag meaning)
predicted_example = dataset['tag'].copy()
predicted_example[0:8] = 'CD'  # Wrong prediction example for the first 8 words
display_pos_tagging(dataset['token'],
                    predicted_example,
                    dataset['tag'],
                    limit=120)

### 1.4 - Pre-processing
Our dataset is already relatively clean; however, one point that might be worth considering is how to handle lowercase conversions. Some tokens in our dataset will be intrinsically capitalized (e.g. proper nouns, the personal pronoun "I"), whereas some other will be capitalized only because they follow a period in the sentence they occur in.

One might think of converting a token to lowercase based on its tag (e.g. if a token is a proper noun, keep it capitalized); however, to be fair, this could only be done on the training set, since in a real scenario test-set tags would be unknown.

Anyway, all these considerations hold only if GloVe contains embeddings of capitalized words; if that's not the case, every word we keep as capitalized will be classified as OOV when matched with GloVe, even when their lowercase embedding actually exists.

As it turns out, Glove does not encode capitalized words:


In [ ]:
num_capitalized = len(
    list(filter(lambda w: w[0].isupper(), glove.key_to_index.keys())))

print(f'GloVe-50 encodes {num_capitalized} capitalized words')

Therefore, we will be forced to convert all tokens to lowercase.

We are also interested to see which "special" tokens are encoded in GloVe, i.e. punctuation, quotation marks, and tokens such as "-LRB-" and "-RRB-", which in our dataset replace "(" and ")", respectively.

As it turns out, GloVe contains every special symbol we care about, except for tokens reserved to brackets:

In [ ]:
special_tokens = [
    *',.:;"`$#£!%/?^-()[]{}_', "''", "``", "--", "-LRB-", "-RRB-", "-LSB-",
    "-RSB-", "-LCB-", "-RCB-"
]
for st in special_tokens:
    if st not in glove:
        print(f"GloVe does not contain token {st}")

Based on the previous considerations, we convert all tokens to lowercase and replace "-LRB"-like symbols with the corresponding bracket:

In [ ]:
# Convert the brackets
for token, bracket in [('-LRB-', '('), ('-RRB-', ')'), ('-LSB-', '['),
                       ('-RSB-', ']'), ('-LCB-', '{'), ('-RCB-', '}')]:
    dataset.loc[dataset.token == token, 'token'] = bracket

# Convert dataset tokens to lowercase
dataset.loc[:, 'token'] = dataset['token'].str.lower()

### 1.5 - Splitting
After pre-processing the data, we can split the dataset into train, validation and test sets.

In [ ]:
TRAIN_DOC_UB = 99
TEST_DOC_LB = 150

ds_train = dataset[dataset['document'].le(TRAIN_DOC_UB)]
ds_val = dataset[dataset['document'].between(
    TRAIN_DOC_UB, TEST_DOC_LB, inclusive='neither')].reset_index()
ds_test = dataset[dataset['document'].ge(TEST_DOC_LB)].reset_index()

print_split = lambda df: f"{df.groupby('document').ngroups} documents, {len(df)} tokens"
print(f"""Dataset split: 
    TRAIN: {print_split(ds_train)}
    VALIDATION: {print_split(ds_val)}
    TEST: {print_split(ds_test)}
""")

### 1.6 - OOV Handling

#### 1.6.1 - OOV Analysis
First of all, let us take a look at how many Out-Of-Vocabulary tokens (w.r.t. GloVe) our dataset contains. In order to simulate a real-world scenario, in which test samples are not readily available at training time, we are going to check (and then handle) OOVs *incrementally*; that is, we will consider:
* **OOV1:** training-set tokens which are not found in V1 = GloVe.
* **OOV2:** validation-set tokens which are not found in V2 = `union(`V1, OOV1`)`.
* **OOV3:** test-set tokens which are not found in V3 = `union(`V2, OOV2`)`.

Our final vocabulary, which will provide encodings to our model(s), is V4 = `union(`V3, OOV3`)` = `union(`GloVe, OOV1, OOV2, OOV3).

In [ ]:
glove_keys = set(glove.key_to_index.keys())
oov1 = set(ds_train['token']) - glove_keys
v2 = glove_keys.union(oov1)
oov2 = set(ds_val['token']) - v2
v3 = v2.union(oov2)
oov3 = set(ds_test['token']) - v3

print(f'OOV1:  {len(oov1)} tokens')
print(f'OOV2:  {len(oov2)} tokens')
print(f'OOV3:  {len(oov3)} tokens')
l = len(oov1) + len(oov2) + len(oov3)
print(
    f'Total: {l} tokens ({l / len(set(dataset["token"])) * 100:.2f}% of dataset)'
)


#### 1.6.2 - Adding OOVs to GloVe
We can now add OOV tokens to the GloVe vocabulary. Many strategies can be adopted to encode OOVs as vectors:
1. Static embeddings with the same vector for all OOV tokens (e.g. zeros).
2. Random embeddings. 
3. Computing an embedding as some statistic involving neighboring tokens (e.g. their mean).

Two observations can guide us in the choice of an embedding strategy:
* OOV tokens are not negligible (about 6% of the *total* dataset).
* Our GloVe embeddings will not undergo further training, therefore fixed or random embedding values will not be refined during the training process.

For the two reasons above, given an OOV token, we will compute its embedding as the mean of its left and right neighbors across all its occurrences throughout the dataset:


In [ ]:
def compute_neighbor_mean(oov_token: str, df: pd.DataFrame,
                          embeddings: KeyedVectors) -> np.ndarray:
    # Find indexes where the oov token appears, and shift them by -1 +1
    indexes = df.index[df['token'] == oov_token].values
    indexes = np.concatenate((indexes - 1, indexes + 1))

    # For each oov word index, look at the left and right until a word with embedding has been found
    neighbor_embeddings = []
    for idx in indexes:
        for direction in (range(idx - 1, -1, -1), range(idx + 1, len(df))):
            for i in direction:
                tok = df['token'].iloc[i]
                if tok not in embeddings:
                    continue
                vector = embeddings[tok]
                neighbor_embeddings.append(vector)
                break

    return np.mean(neighbor_embeddings, axis=0)


def add_oovs(oov_tokens: Set, df: pd.DataFrame,
             embeddings: KeyedVectors) -> KeyedVectors:
    # Clone the embedding (KeyedVectors does not have a clone method)
    emb_filled = deepcopy(embeddings)

    # Estimate the OOV embeddings
    keys, values = [], []
    for oov in oov_tokens:
        vector = compute_neighbor_mean(oov, df, emb_filled)
        keys.append(oov)
        values.append(vector)
    # Add the estimates to the embedding
    emb_filled.add_vectors(keys, values)
    return emb_filled

In [ ]:
# V2 = union(Glove, OOV1)
# where neighbors of OOV1 are taken from the trainig set
embeddings = add_oovs(oov1, ds_train, glove)

# V3 = union(V2, OOV2)
# where neighbors of OOV2 are taken from the validation set
embeddings = add_oovs(oov2, ds_val, embeddings)

# V4 = union(V3, OOV3)
# where neighbors of OOV3 are taken from the test set
embeddings = add_oovs(oov3, ds_test, embeddings)

# Test number of embeddings
print(f'Number of vectors in original GloVe:                  {len(glove)}')
print(
    f'Number of vectors after incremental addition of OOVs: {len(embeddings)}')


### 1.7 - Embedding Matrix

In [ ]:
# Define the indexes used for word -> index -> embedding
word_index = {k: v + 1
              for k, v in embeddings.key_to_index.items()
              }  # +1 because index 0...
vocab_size = len(embeddings) + 1  # ...will be reserved to padding

# Define the embedding matrix
embedding_matrix = np.zeros(shape=(vocab_size, EMB_DIM))
for word, index in word_index.items():
    embedding_matrix[index] = embeddings[word]

In [ ]:
# Quick test
assert np.all(embedding_matrix[embeddings.key_to_index['cat'] +
                               1] == embeddings['cat'])

### 1.8 - Data Conversion

The input data of our model could be either whole documents or single sentences contained in those documents; we will choose sentences as input data.

Tokens in each sentence will be converted to integer sequences and later fed into a static `Embedding` layer storing the matrix of Glove encodings + OOVs, which will provide the input to our model.

The corresponding tags—i.e. the output of our model—will be instead one-hot encoded. The rationale behind this choice is that tags are purely categorical data, hence encoding them as integer sequences would inject a notion of ordering into the model, which however is not reflected in the original data.

In [ ]:
# Utility function
flatten_1d = lambda nested_list: [li[0] for li in nested_list]

# Convert tokens into sequences (their vocabulary indexes)
tokenizer = Tokenizer(filters='')
tokenizer.word_index = word_index
token_indexes = tokenizer.texts_to_sequences_generator(dataset['token'].array)
token_indexes = flatten_1d(token_indexes)

# Convert tags into sequences
# (as an intermediate step before one-hot encoding them)
tag_to_int = {k: v + 1 for v, k in enumerate(dataset['tag'].unique())}
num_tags = len(tag_to_int) + 1
tokenizer = Tokenizer(filters='', lower=False)
tokenizer.word_index = tag_to_int
tag_indexes = tokenizer.texts_to_sequences_generator(dataset['tag'].array)
tag_indexes = flatten_1d(tag_indexes)

# Augment dataset with new data
dataset['token_index'] = token_indexes
dataset['tag_index'] = tag_indexes

# Group dataset by 'sentence', aggregating remaining data into lists
ds_sentences = dataset.groupby(['document', 'sentence']).agg(list)
ds_sentences.head()

Let's take a look at the distribution of sentence length in the training + validation set (leaving the test set aside) to determine what is an appropriate padded-sequence size (for batching).

In [ ]:
sentences_len = ds_sentences.query(
    f'document <= {TEST_DOC_LB-1}')['token'].transform(len)
len_quantile = sentences_len.quantile(.99)
print("99th percentile of sentence length in training + validation set:",
      len_quantile)

fig = px.histogram(sentences_len, labels={'value': 'Sentence Length'})
fig.add_vline(len_quantile,
              annotation_text="99th percentile",
              line_color='green')
fig.update_layout(showlegend=False)
fig.show()

The 99th percentile suggests to trim sentences that exceed 56 tokens and pad sentences with fewer tokens, in order to prevent the few outliers from causing sentence encodings to be wastefully long.  We choose pre-padding, as it has been experimentally proven to be more effective than its counterpart post-padding.
> TODO: visto che vogliamo fare così i raffinati, a sto punto citiamo il paper come si deve.

In [ ]:
pad = lambda x: pad_sequences(
    x, maxlen=int(len_quantile), padding='pre', truncating='pre')


def get_model_data(df: pd.DataFrame, lb=0, ub=None):
    df = df.query(f'{lb} <= document <= {ub}') if ub else df.query(
        f'{lb} <= document')
    toks = pad(df['token_index'])
    tags = pad(df['tag_index'])
    # One-hot encode tags
    tags = to_categorical(tags, num_classes=num_tags)
    return toks, tags


# Build the data that will be fed to the model
x_train, y_train = get_model_data(ds_sentences, 0, TRAIN_DOC_UB)
x_val, y_val = get_model_data(ds_sentences, TRAIN_DOC_UB + 1, TEST_DOC_LB - 1)
x_test, y_test = get_model_data(ds_sentences, TEST_DOC_LB)

# Check shapes
print(f"""
X shapes [sentences x tokens]
    x_train.shape = {x_train.shape}
    x_val.shape   = {x_val.shape}
    x_test.shape  = {x_test.shape}

Y shapes [sentences x tags x one-hot-size]
    y_train.shape = {y_train.shape}
    y_val.shape   = {y_val.shape}
    y_test.shape  = {y_test.shape}
""")

## 2 - Model Definition

### 2.2 - Models and parameters

In [ ]:
# Hyperparameters shared by all models models
model_hyperparams = dict(
    epochs=2000,
    batch_size=512,
    mem_units=32,
    optimizer='adam',
    loss='categorical_crossentropy',
)

# Embedding layer hyperparameters — same for all models
embedding_hyperparams = dict(
    input_dim=vocab_size,
    output_dim=EMB_DIM,
    input_length=x_train.shape[-1],
    weights=[embedding_matrix],
    trainable=False,
)

In [ ]:
mu = model_hyperparams['mem_units']  # Number of memory units of LSTM / GRU layers

# Baseline: Bidirectional LSTM + Dense layer 
bi_lstm = Sequential([
    Embedding(**embedding_hyperparams),
    Bidirectional(LSTM(mu, return_sequences=True)),
    Dense(num_tags, activation='softmax'),
], "BiLSTM + Dense")

# Bidirectional GRU + Dense layer
bi_gru = Sequential([
    Embedding(**embedding_hyperparams),
    Bidirectional((GRU(mu, return_sequences=True))),
    Dense(num_tags, activation='softmax'),
], "BiGRU + Dense")

# Bidirectional LSTM + Bidirectional LSTM + Dense layer
double_bi_lstm = Sequential([
    Embedding(**embedding_hyperparams),
    Bidirectional(LSTM(mu, return_sequences=True)),
    Bidirectional(LSTM(mu, return_sequences=True)),
    Dense(num_tags, activation='softmax'),
], "Double-BiLSTM + Dense")

# Bidirectional LSTM + Dense layer + Dense layer
double_dense = Sequential([
    Embedding(**embedding_hyperparams),
    Bidirectional(LSTM(mu, return_sequences=True)),
    Dense(2 * num_tags, activation='relu'),
    Dense(num_tags, activation='softmax'),
], " BiLSTM + Double-Dense")

## 3 - Training (and validation)

In [ ]:
for model in [
        bi_lstm, bi_gru, double_bi_lstm, double_dense
]:
    # Initialize wandb with relevant info
    config = model_hyperparams
    config['model-name'] = model.name
    run_name = model.name
    run = wandb.init(
        anonymous="must",
        project="NLP-POS-Tagging",
        name=run_name,
        reinit=True,
        config=config,
    )

    # Compile
    model.compile(optimizer=config['optimizer'],
                  loss=config['loss'],
                  metrics=['acc'])
    print(model.summary())

    # Fit on the data and run the validation
    with run:
        history = model.fit(x_train,
                            y_train,
                            epochs=config['epochs'],
                            batch_size=config['batch_size'],
                            validation_data=(x_val, y_val),
                            callbacks=[WandbCallback(save_model=False)])

## 4 - Evaluation

In [ ]:
"""
def to_dataframe(x, y, p) -> pd.DataFrame:
    # Define a mapping int->token and int->tag
    int_to_tok = {v: k for k, v in word_index.items()}
    int_to_tag = {v: k for k, v in tag_to_int.items()}
    convert_tok = np.vectorize(int_to_tok.get)
    convert_tag = np.vectorize(lambda i: int_to_tag.get(i, 'PAD'))

    # Convert one-hot encodings to integer indexes
    y = np.argmax(y, axis=2)
    p = np.argmax(p, axis=2)

    # Convert integer indexes to the corresponding token/tag, skipping padding
    raw_df = dict(token=[], tag=[], tag_predicted=[])
    for zx, zy, zp in zip(x, y, p):
        pad_mask = zx > 0
        raw_df['token'] += convert_tok(zx[pad_mask]).tolist()
        raw_df['tag'] += convert_tag(zy[pad_mask]).tolist()
        raw_df['tag_predicted'] += convert_tag(zp[pad_mask]).tolist()

    # Note: the returned df will contain truncated sentences
    return pd.DataFrame(raw_df)


p_val = model.predict(x_val)
ds_val_pred = to_dataframe(x_val, y_val, p_val)
ds_val_pred.head()
"""

In [ ]:
"""
display_pos_tagging(ds_val_pred['token'],
                    ds_val_pred['tag'],
                    ds_val_pred['tag_predicted'],
                    limit=120)
"""

In [ ]:
"""
def plot_confusion_matrix(df: pd.DataFrame):
    corr, pred = df['tag'], df[
        'tag_predicted']  # Extract true tags and predictions
    labels = list(set(corr) | set(pred))  # Get the tag names

    # Compute confusion matrix
    conf = confusion_matrix(corr, pred, labels=labels, normalize='true')

    # Plot matrix
    fig = px.imshow(conf,
                    x=labels,
                    y=labels,
                    labels={
                        'x': 'Predicted Tag',
                        'y': 'True Tag',
                        'color': 'Value'
                    })
    fig.update(layout_coloraxis_showscale=False)
    fig.update_layout(width=600, height=600)
    fig.show()


plot_confusion_matrix(ds_val_pred)
"""